In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../datasets/cleaned_cmts_voo.csv')
df.head()

,author,id,date,permalink,body,score,subreddit
0,investing_me,kwsc2e7,2024-03-27,/r/ETFs/comments/1boyqg6/375qqq125tqqq375voo12...,Just buy VOO at this point,6,ETFs
1,iamfredrick,kwv4b7y,2024-03-27,/r/ETFs/comments/1bpdnmd/schg_schd_vs_voo/kwv4...,Voo & schg have 53% overlap. Past results don’...,3,ETFs
2,Fun_Grapefruit_3416,kwvj6cq,2024-03-27,/r/ETFs/comments/1bpdnmd/schg_schd_vs_voo/kwvj...,Schd + Schg makes up the majority of voo. Just...,2,ETFs
3,rem14,kwvnzn8,2024-03-27,/r/ETFs/comments/1bpdnmd/schg_schd_vs_voo/kwvn...,13% overlap with VOO and less than 1% overlap ...,2,ETFs
4,DITPiranha,kx0qfv5,2024-03-28,/r/ETFs/comments/1bq2hk5/voo_vgt_vti/kx0qfv5/,"30% VOO, 30% QQQ, 10% VYM, 10% BND, 10% Mixed ...",1,ETFs


In [3]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [4]:
sentence_pos = "Stock price of VOO will go higher."
sentence_neg = "Never ever buy VOO"

In [5]:
classifier(sentence_pos), classifier(sentence_neg)

([{'label': 'POSITIVE', 'score': 0.5952612161636353}],
 [{'label': 'NEGATIVE', 'score': 0.9967347979545593}])

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = 'distilbert-base-uncased-finetuned-sst-2-english'
pt_model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [8]:
inputs_pos, inputs_neg = tokenizer(sentence_pos), tokenizer(sentence_neg)

In [9]:
inputs_pos, inputs_neg

({'input_ids': [101, 4518, 3976, 1997, 29536, 2080, 2097, 2175, 3020, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]},
 {'input_ids': [101, 2196, 2412, 4965, 29536, 2080, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]})

In [10]:
tokenizer.convert_ids_to_tokens([29536, 2080])

['vo', '##o']

In [11]:
pt_batch = tokenizer(
    [sentence_pos, sentence_neg], 
    padding=True, 
    truncation=True,
    max_length=512,
    return_tensors="pt"
)

In [12]:
pt_batch

{'input_ids': tensor([[  101,  4518,  3976,  1997, 29536,  2080,  2097,  2175,  3020,  1012,
           102],
        [  101,  2196,  2412,  4965, 29536,  2080,   102,     0,     0,     0,
             0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}

In [13]:
pt_outputs = pt_model(**pt_batch)

pt_outputs

SequenceClassifierOutput(loss=None, logits=tensor([[-0.1126,  0.2732],
        [ 3.1458, -2.5754]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)